In [ ]:
%matplotlib inline
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

In [ ]:
os.listdir('../input/')

In [ ]:
PATH = '../input/train.csv'
train = pd.read_csv(PATH)
train.head()

In [ ]:
train.describe()

In [ ]:
train.isnull().sum()

**Let's do some Data Visualisation**

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
sns.distplot(train["count"], ax=ax, hist=True, kde=True)

Data is very much skewed. Take log needs to be done.

In [ ]:
# log transformation, add 1 because some values are close to 0
y = np.log1p(train["count"])

In [ ]:
sns.distplot(y, kde=True)
plt.show()

**Feature Analysis**

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20, 4))
sns.distplot(train.temp, kde=True, color='green', ax=ax[0])
sns.distplot(train.atemp, kde=True, color='green', ax=ax[1])
sns.distplot(train.humidity, kde=True, color='green', ax=ax[2])
sns.distplot(train.windspeed, kde=True, color='green', ax=ax[3])
plt.show()

We can see in the third plot that there are a lot of zeros for windspeed which seems rather strange. Let's analyse it rather closely.

There are about 1500 zeros in the "windspeed" feature column.

In [ ]:
train['windspeed'].describe()

In [ ]:
# datetime feature
train["date"] = train.datetime.apply(lambda x : x.split()[0])
train["hour"] = train.datetime.apply(lambda x : x.split()[1].split(":")[0]).astype("int")
train["year"] = train.datetime.apply(lambda x : x.split()[0].split("-")[0])
train["weekday"] = train.date.apply(lambda dateString : datetime.strptime(dateString,"%Y-%m-%d").weekday())
train["month"] = train.date.apply(lambda dateString : datetime.strptime(dateString,"%Y-%m-%d").month)

In [ ]:
train.head()

In [ ]:
train_X = train.drop(['casual','registered','datetime', 'date', 'year', 'atemp'],axis=1)
train_y = train['count']
train_X.head()

In [ ]:
train_X = train_X.drop(['count'], axis=1)

In [ ]:
train_X = train_X.values

In [ ]:
from sklearn.ensemble import RandomForestRegressor
'''
from sklearn.grid_search import GridSearchCV

estimators = list(range(50, 100, 10))
sam_leaf = list(range(90, 101, 10))
param_grid = dict(n_estimators=estimators, min_samples_leaf=sam_leaf)
print(param_grid)

rf = RandomForestRegressor()
grid = GridSearchCV(rf, param_grid, cv=10, scoring='neg_mean_squared_error')
grid.fit(train_X, train_y)
print(grid.best_score_)
print(grid.best_params_)
'''

In [ ]:
model = RandomForestRegressor(n_estimators=120, min_samples_leaf=18)
model.fit(train_X, train_y)
test_file_path = '../input/test.csv'
test = pd.read_csv(test_file_path)
test.head()

In [ ]:
test["date"] = test.datetime.apply(lambda x : x.split()[0])
test["hour"] = test.datetime.apply(lambda x : x.split()[1].split(":")[0]).astype("int")
test["year"] = test.datetime.apply(lambda x : x.split()[0].split("-")[0])
test["weekday"] = test.date.apply(lambda dateString : datetime.strptime(dateString,"%Y-%m-%d").weekday())
test["month"] = test.date.apply(lambda dateString : datetime.strptime(dateString,"%Y-%m-%d").month)

In [ ]:
test.head()

In [ ]:
# Replace the zeros with mean
test["windspeed"] = test["windspeed"].replace(0.0, train['windspeed'].mean())

In [ ]:
test_X = test.drop(['datetime', 'date', 'year', 'atemp'],axis=1)
test_X.head()

In [ ]:
prediction = model.predict(test_X)
print(prediction)

In [ ]:
sub = pd.DataFrame({'datetime': test.datetime, 'count': prediction})
sub.to_csv('submission.csv', index=False)